## Brief Description
This notebook is used for creating the folder structure and data necessary for the model by recording 30 frames, 30 times, for a given label.

In [5]:
from ADUtils import *

# Folder Setup

In [3]:
create_extra_sequences = True # just so you do not accidently create new folders

# this block creates $no_sequences folders for each action in DATA_PATH for recording
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)
if create_extra_sequences:
    for action in actions: # if you only want to create folders for a certain action, comment this out and uncomment below
#         action = "test"
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            os.mkdir(action_path)
        dir_max = 0
        # if path is not empty find the maximum folder number and start creating from max
        if os.listdir(action_path): 
            dir_max = np.max(np.array(os.listdir(action_path)).astype(int))+1
        for sequence in range(0, no_sequences):
            try:
                os.makedirs(os.path.join(action_path, str(dir_max+sequence)))
            except:
                pass

In [4]:
DELETE_EMPTY_FOLDERS = False

if DELETE_EMPTY_FOLDERS:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            continue
        
        for folder in os.listdir(action_path):
            if not os.listdir(os.path.join(action_path, folder)):
                os.rmdir(os.path.join(action_path, folder))

# Data Creation

Run the cell below with the action you want, word for word from the list
Then run the cell below that to save your data

In [ ]:
actions

In [ ]:
action = 'raise_hand' # choose which action you want data for from list above

Step 1: Select an action from the list above by rewriting action variable

Step 2: Run the cell below and record yourself performing the action

Step 3: Repeat Step 1 until you completed all actions

In [ ]:
cap = cv2.VideoCapture(0)

# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     for action in actions: # if you want to loop through all actions

    # Find maximum sequence for action
    action_path = os.path.join(DATA_PATH, action)
    start_sequence = 9999 # if you have more data than this than change it to be great or sys.maxint
    for folder in os.listdir(action_path):
        if not os.listdir(os.path.join(action_path, folder)):
            if int(folder) < start_sequence:
                start_sequence = int(folder)
    
    for sequence in range(start_sequence, start_sequence+no_sequences):
        # Loop through video length aka sequence length
        for frame_num in range(sequence_length):

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}/{}'.format(action, sequence, start_sequence+no_sequences), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                cv2.waitKey(500)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}/{}'.format(action, sequence, start_sequence+no_sequences), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)


            cv2.imshow('OpenCV Feed', image)
                
            

            # Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
    cap.release()
    cv2.destroyAllWindows()
# need it? who knows
cap.release()
cv2.destroyAllWindows()